In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import curve_fit
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def return_language(lang):
    if lang == "en": return "English"
    elif lang == "fr": return "French"
    elif lang == "fi": return "Finish"
    elif lang == "de": return "German"
    elif lang == "it": return "Italian"
    elif lang == "nl": return "Dutch"
    elif lang == "es": return "Spanish"
    elif lang == "pt": return "Portuguese"
    elif lang == "hu": return "Hungarian"
    elif lang == "sv": return "Swedish"
    elif lang == "eo": return "Esperanto"
    elif lang == "la": return "Latin"
    elif lang == "da": return "Danish"
    elif lang == "tl": return "Tagalog"
    elif lang == "ca": return "Catalan"
    elif lang == "pl": return "Polish"
    elif lang == "no": return "Norwegian"
    elif lang == "cs": return "Czech"
    elif lang == "cy": return "Welsh"
    elif lang == "is": return "Icelandic"
    elif lang == "af": return "Afrikaans"


def func_powerlawexp(x, alfa, beta, gamma):
        return alfa * x**(beta) * np.exp(-gamma*x)
    
def latex_float(f):
    float_str = "{:.1e}".format(f)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return "${0} \\cdot 10^{{{1}}}$".format(base, int(exponent))
    else:
        return float_str

In [ ]:
listaidiomas = ["en", "fr", "fi", "de", "it", "nl", "es", "pt", "hu", "sv", "eo", "la", "da",
                 "tl", "ca", "pl", "no", "cs", "cy", "is", "af"]

# 0. Spearman Correlations without first

In [ ]:
list2save = []
for lang in listaidiomas:
    outputname = "MAL_FullFrom2_Spearman" + lang
            
    result_path = "results/" + lang + "/menzerath_altmann_" + lang + ".csv"
    df = pd.read_csv(result_path, index_col=0, header=[0,1]).mean_syllabe_length
    df = df[df["count"]>25]
#     df = df[df.index.values<13] # At least some values FILTER
    df = df[df.index.values>1]
    correlationdata, pvaluedata = stats.spearmanr(df["mean"].index, df["mean"].values)

    BHMM_path = "results/" + lang + "/HMM_menzerath_" + lang + ".csv"
    BHMM = pd.read_csv(BHMM_path, index_col=1)["value"]
    
    correlationBHMM, pvalueBHMM = stats.spearmanr(BHMM.index, BHMM.values)

    
    list2save.append([return_language(lang) + " & " + str(round(correlationdata,2)) + " & " + str(round(pvaluedata,2)) +
                      " & " + str(round(correlationBHMM, 2)) + " & " + str(round(pvalueBHMM,2)) + "\\" + "\\"])

pd.DataFrame(list2save, columns=['Language & Data corr & Data p & BHMM corr & BHMM p']).to_csv("results/Figures_Article/FullCorpusFrom2/Correlations.csv", index=False)

# 0. Spearman Correlationsfull

In [ ]:
list2save = []


for lang in listaidiomas:
    outputname = "MAL_FullFrom2_Spearman" + lang
            
    result_path = "results/" + lang + "/menzerath_altmann_" + lang + ".csv"
    df = pd.read_csv(result_path, index_col=0, header=[0,1]).mean_syllabe_length
    df = df[df["count"]>25]
#     df = df[df.index.values<13] # At least some values FILTER
#     df = df[df.index.values>1]
    correlationdata, pvaluedata = stats.spearmanr(df["mean"].index, df["mean"].values)

    BHMM_path = "results/" + lang + "/HMM_menzerath_" + lang + ".csv"
    BHMM = pd.read_csv(BHMM_path, index_col=1)["value"]
    
    correlationBHMM, pvalueBHMM = stats.spearmanr(BHMM.index, BHMM.values)

    
    list2save.append([return_language(lang) + " & " + str(round(correlationdata,2)) + " & " + str(round(pvaluedata,2)) +
                      " & " + str(round(correlationBHMM, 2)) + " & " + str(round(pvalueBHMM,2)) + "\\" + "\\"])

pd.DataFrame(list2save, columns=['Language & Data corr & Data p & BHMM corr & BHMM p']).to_csv("results/Figures_Article/FullCorpusFrom1/Correlations.csv", index=False)

In [ ]:
def plot_Menzerath(f, ax, outputname, flag, lang, dataframe, HMM, output_dir, from1=None):
    if flag%5==0: color = "blue"
    if flag%5==1: color = "red"
    if flag%5==2: color = "green"
    if flag%5==3: color = "orange"
    if flag%5==4: color = "purple"
    if flag==21: color = "gray"
        
    language = return_language(lang)
    syllables = dataframe.index.values
    phonemes = dataframe["mean"].values


    # # MAIN PLOT
    ax.plot(syllables, phonemes, 'o', lw = 2, ms = 8, zorder=2, alpha=1, color="darkblue", label=language)
    
    x = syllables
    y = phonemes
    xlarge = np.arange(syllables[0], syllables[-1], 0.1)

    # # POWER LAW EXPONENTIAL
    popt, pcov = curve_fit(func_powerlawexp, syllables, phonemes)
    expy = func_powerlawexp(xlarge, popt[0], popt[1], popt[2])
    ax.plot(xlarge, expy, "--", lw=1.5, color="darkred", alpha=0.9, label="fit to " + r"$y=\alpha x^{\beta}e^{-\gamma}$")

    # # GET R2
    residuals = y - func_powerlawexp(x, popt[0], popt[1], popt[2])
    ss_res = np.sum(residuals**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    R2 = 1 - (ss_res / ss_tot)
    
    parametros = [language + " & " + str(round(popt[0],2)) + " & " + latex_float(popt[1])+ " & " + latex_float(popt[2])+ 
                  " & " + str(round(R2,2))+ " & " + str(round(popt[1]/popt[2], 1)) + "\\" + "\\"]
    
    ax.plot(HMM.index, HMM.values, '-', lw = 4, ms = 1, zorder=2, alpha=0.6, color="gray", label= "BHMM")

    ax.legend(frameon = True, loc = "upper right", fontsize=10, ncol= 3)

    ax.set_xticks([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
    ax.set_yticks([1.8, 2.1, 2.4, 2.7, 3.0, 3.3])
    ax.set_ylim([1.8,3.4])
    ax.set_xlim([1.8, 12.2])
    
    if lang=="it" or lang=="nl" or lang=="es" or lang=="pt":
        ax.set_xticks([2, 3, 4, 5, 6, 7, 8, 9])
        ax.set_yticks([1.9, 2.2, 2.5, 2.8, 3.1])
        ax.set_ylim([1.8,3.2])
        ax.set_xlim([1.8, 9.2])
    
    if from1:
        ax.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
        ax.set_xlim([0.8, 11.2])
        

    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    ax.set_xlabel(r"$Word$" + " " + r"$\overline{size}$" + " " + r"($number$" + " " + r"$of$" + " " + r"$syllables)$", fontsize=12)
    ax.set_ylabel(r"$Syllable$" + " " + r"$\overline{size}$" + " " + r"$(characters)$", fontsize=12)

    plt.tight_layout()
    f.savefig(output_dir + outputname + ".pdf", bbox_inches = 'tight', pad_inches = 0.1)
    return(parametros)

# 1. MAL full corpus starting from words with TWO syllables

In [ ]:
flag = 0
parameterstosave = []

for lang in listaidiomas:
    flag +=1

    f, ax = plt.subplots()
    outputname = "MAL_FullFrom2_" + lang
            
    result_path = "results/" + lang + "/menzerath_altmann_" + lang + ".csv"
    df = pd.read_csv(result_path, index_col=0, header=[0,1]).mean_syllabe_length
    df = df[df["count"]>25]
#     df = df[df.index.values<13] # At least some values FILTER
    
    BHMM_path = "results/" + lang + "/HMM_menzerath_" + lang + ".csv"
    BHMM = pd.read_csv(BHMM_path, index_col=1)["value"]

    output_dir = "results/Figures_Article/FullCorpusFrom2/"
    df_withoutfirst = df[2::]
    parameterstosave.append(plot_Menzerath(f, ax, outputname, flag, lang, df_withoutfirst, BHMM[1:int(df.index.values[-1])], output_dir))
    
pd.DataFrame(parameterstosave, columns=['Language & alpha & beta & gamma & R2']).to_csv("results/Figures_Article/FullCorpusFrom2/parametersFullCorpusFrom2.csv", index=False)

# 2. MAL full corpus starting from words with one syllable


In [ ]:
parameterstosave = []
flag = 0
for lang in listaidiomas:
    flag +=1
    f, ax = plt.subplots()
    outputname = "MAL_FullFrom1_" + lang
            
    result_path = "results/" + lang + "/menzerath_altmann_" + lang + ".csv"
    df = pd.read_csv(result_path, index_col=0, header=[0,1]).mean_syllabe_length
    df = df[df["count"]>25] # At least some values FILTER
#     df = df[df.index.values<12] # At least some values FILTER
    
    BHMM_path = "results/" + lang + "/HMM_menzerath_" + lang + ".csv"
    BHMM = pd.read_csv(BHMM_path, index_col=1)["value"]

    output_dir = "results/Figures_Article/FullCorpusFrom1/"
    parameterstosave.append(plot_Menzerath(f, ax, outputname, flag, lang, df, BHMM[0:int(df.index.values[-1])], output_dir, from1=True))

pd.DataFrame(parameterstosave, columns=['language & alpha & beta & gamma & R2 & beta/gamma']).to_csv("results/Figures_Article/FullCorpusFrom1/parametersFullCorpusFrom1.csv", index=False)


# 3. MAL full corpus vs averaging by books


In [ ]:
for lang in listaidiomas:
    output_dir = "results/Figures_Article/Compare/"
    f, ax = plt.subplots()

    # MAL AVERAGED BY BOOK
    resultsAverage = "results/" + lang + "/MALbyBook_" + lang + ".csv"
    df = pd.read_csv(resultsAverage, header=None, sep = " ")

    # PLOT MAL FOR SOME BOOKS
    for i in range(len(df)):
        if i>500:
            break
        a = df.loc[i]
        a.index+=1
        a=a[a>0]
        if i==0:
            ax.plot(a.index.values[0], a.values[0], '-', lw = 1, ms = 1, zorder=0, alpha=1, color="gray", label="MAL for each book")
        else:
            ax.plot(a.index.values, a.values, '-', lw = 1, ms = 1, zorder=2, alpha=0.1, color="gray")


    # AVERAGE BY BOOK
    lista_syllablesByBook = []
    lista_meanvalueByBook = []
    for i in range(11):
        toconsider = df[i]
        greaterthanzero = toconsider[toconsider>0]
        if len(greaterthanzero)>1:
            lista_syllablesByBook.append(i+1)
            lista_meanvalueByBook.append(greaterthanzero.mean())
        else:
            break

    # FULL DATA
    resultFULL = "results/" + lang + "/menzerath_altmann_" + lang + ".csv"
    dfFULL = pd.read_csv(resultFULL, index_col=0, header=[0,1]).mean_syllabe_length
    dfFULL = dfFULL[dfFULL["count"]>25] # At least some values FILTER
    dfFULL = dfFULL[dfFULL.index.values<12] # At least some values FILTER


    # PLOT
    if lang == "fr":
        ax.plot(lista_syllablesByBook[0:len(dfFULL.index.values)-2], lista_meanvalueByBook[0:len(dfFULL.index.values)-2], 
                'o-', lw = 2, ms = 10, zorder=2, alpha=0.8, color="black", label="MAL averaged by book")
        ax.plot(dfFULL.index.values[0:-2], dfFULL["mean"].values[0:-2], 
                's-', lw = 2, ms = 10, zorder=2, alpha=0.8, color="darkblue", label="MAL for full corpus")
    else:    
        ax.plot(lista_syllablesByBook[0:len(dfFULL.index.values)-1], lista_meanvalueByBook[0:len(dfFULL.index.values)-1], 
                'o-', lw = 2, ms = 10, zorder=2, alpha=0.8, color="black", label="MAL averaged by book")
        ax.plot(dfFULL.index.values[0:-1], dfFULL["mean"].values[0:-1], 
                's-', lw = 2, ms = 10, zorder=2, alpha=0.8, color="darkblue", label="MAL for full corpus")

    ax.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
    ax.set_xlim([0.8, 10.2])
    
    if lang=="de":
        ax.set_yticks([2.7, 3.0, 3.3])
        ax.set_ylim([2.5,3.4])
        
    elif lang=="en":
        ax.set_xticks([1, 2, 3, 4, 5, 6, 7])
        ax.set_xlim([0.8, 7.2])
        ax.set_ylim([2.1,3.2])
        
    elif lang=="fr":
        ax.set_xticks([1, 2, 3, 4, 5, 6, 7])
        ax.set_xlim([0.8, 7.2])
        ax.set_ylim([2.3,2.9])

    elif lang=="fi":
        ax.set_xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
        ax.set_xlim([0.8, 10.2])
        ax.set_ylim([2.4,2.85])
        
    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)

    ax.set_xlabel(r"$Word$" + " " + r"$\overline{size}$" + " " + r"($number$" + " " + r"$of$" + " " + r"$syllables)$", fontsize=12)
    ax.set_ylabel(r"$Syllable$" + " " + r"$\overline{size}$" + " " + r"$(characters)$", fontsize=12)

    leg = ax.legend(frameon = True, loc = "best", fontsize=10, ncol= 1, title=return_language(lang))
    leg.get_frame().set_linewidth(0.0)
    leg.get_title().set_fontsize('13')


    plt.tight_layout()

    f.savefig(output_dir + "FullvsAverage_" + lang + ".pdf", bbox_inches = 'tight', pad_inches = 0.1)


# 4. Clustersizes representation


In [ ]:
for lang in listaidiomas:
    output_dir = "results/Figures_Article/ClusterSizes/"
    f, ax = plt.subplots()
    n = 10
    colors = plt.cm.gist_heat(np.linspace(0,1,n))

    # MAL AVERAGED BY BOOK
    resultsAverage = "results/" + lang + "/clustersizes_" + lang + ".csv"
    df = pd.read_csv(resultsAverage, header=None, sep = ",", keep_default_na=True, skiprows=1, index_col=0).fillna(0)

    lista_syllables = []
    minimoy = 5
    maximoy = 0
    for i in range(1,8):
        lista_syllables.append(i)
        if sum(df.loc[i+2]>0)<1: break
        if sum(df.loc[i+1]>0)<1: break
        if lang=="is" and i>6: break
        clusters_n = df.loc[i]
        clusters_n = clusters_n[clusters_n>0].values
        
        if i == 1:
            ax.plot(0.5, clusters_n, 'o-', c=colors[i], ms=8, lw=3, alpha=0.9, label=str(i) + " syll.")
        else:
            if min(clusters_n)<minimoy: minimoy=min(clusters_n)
            if max(clusters_n)>maximoy: maximoy=max(clusters_n)
            array = np.asarray(lista_syllables)
            clustersizes = (array-min(array))/(max(array) - min(array))
            ax.plot(clustersizes, clusters_n, 'o-', c=colors[i], ms=8, lw=2, alpha=0.8, label=str(i) + " syll.")

    ax.set_xticks([0, 0.5, 1])
    ax.set_xlim([-0.03, 1.03])
    ax.set_ylim([minimoy-0.2, maximoy+0.05])

    ax.tick_params(axis='x', labelsize=12)
    ax.tick_params(axis='y', labelsize=12)

    ax.set_xlabel(r"$Adimensional$" + " " + r"$word$" + " " + r"$size$", fontsize=12)
    ax.set_ylabel(r"$Cluster$" + " " + r"$\overline{size}$" + " " + r"$(characters)$", fontsize=12)

    leg = ax.legend(frameon = False, loc = 4, fontsize=10, ncol= 4, title=return_language(lang))
    leg.get_title().set_fontsize('12')
    f.savefig(output_dir + "Clustersizes_" + lang + ".pdf", bbox_inches = 'tight', pad_inches = 0.1)    